This file is used to combine lines of code to make it easier to include them in my Medium blog post.

In [ ]:
import pandas_datareader as dr
from datetime import datetime

# Define start and end date
end = datetime(2020, 7, 9)
start = datetime(2015, 8, 6)

# Create a list of the ticker symbols to be used in this project
tickers = ['AMZN', 'GOOGL', 'JNJ', 'V', 'PG', 'UNH', 'JPM', 'HD', 'VZ', 'NFLX', 'DIS', 'MRK', 'PEP', 'BAC', 'KO', 
           'WMT','CVX', 'ABT', 'AMGN', 'MCD', 'COST', 'NKE', 'PM', 'QCOM', 'LOW', 'BA', 'LMT', 'SBUX', 'UPS', 'CAT']

# Obtain the adjusted closing prices from Yahoo Finance
prices = pd.DataFrame()
for tick in tickers:
    prices[tick] = data.DataReader(tick, data_source='yahoo', start=start, end=end)['Adj Close']
prices.columns = tickers

In [ ]:
# Define a function to generate N number of random portfolios given a DataFrame of returns
def generate_ptfs(returns, N):
    ptf_rs = []
    ptf_stds = []
    for i in range(N):
        # Generate random weights between 0 and 1
        weights = np.random.random(len(returns.columns))
        # Ensure that the weights add up to 1
        weights /= np.sum(weights)
        # Append the portfolio return to the list of portfolio returns
        ptf_rs.append(np.sum(returns.mean() * weights) * 252)
        # Append the portfolio volatility to the list of portfolio volatilities
        ptf_stds.append(np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights))))
    ptf_rs = np.array(ptf_rs)
    ptf_stds = np.array(ptf_stds)
    return ptf_rs, ptf_stds